In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.dates import date2num

from datetime import date, timedelta

import urllib.request
from urllib.error import HTTPError

import re

from enum import StrEnum

from concurrent.futures import ThreadPoolExecutor, as_completed

from collections import defaultdict

In [ ]:
class TrackingFlavor(StrEnum):
    legacy_default = "tracking"
    legacy_fast = "fasttracking"
    acts_default = "actstracking"
    acts_fast = "actsfasttracking"

def get_url(d, flavor):
    date_str = d.strftime("%d-%m-%Y")
    base = f"https://atlaspmb.web.cern.ch/atlaspmb/spot-mon-phase2_recoonly_{flavor}/pages/logs"
    binary = "main-x86_64-el9-gcc13-opt"
    job = f"phase2_recoonly_{flavor}-RAWtoALL"
    if d < date.fromisoformat("2024-03-01"):
        base += "/archive_centos7"
        binary = "main-x86_64-centos7-gcc11-opt"
    return f"{base}/{date_str}-{binary}-{job}"

get_url(date.today(), TrackingFlavor.legacy_default)

In [ ]:
def parse_spot(txt):
    p = re.compile(r"\d\d:\d\d:\d\d PerfMonMTSvc\s+INFO (\w+?)\s+(\d+)\s+(\d+(?:\.\d*)?)\s+(\d+(?:\.\d*)?)\s+(\d+(?:\.\d*)?)\s+(.+)")
    columns = ["step", "count", "cpu", "vmem", "malloc", "component"]
    data = []

    for line in txt.splitlines():
        m = p.match(line)
        if not m:
            continue
        data.append(m.groups())

    result = pd.DataFrame(data, columns=columns)
    result[["count"]] = result[["count"]].astype(int)
    result[["cpu", "vmem", "malloc"]] = result[["cpu", "vmem", "malloc"]].astype(float)
    return result

def load_and_parse_spot(d, flavor):
    url = get_url(d, flavor)
    f = urllib.request.urlopen(url)
    txt = f.read().decode("utf-8")
    return parse_spot(txt)

load_and_parse_spot(date.fromisoformat("2024-05-26"), TrackingFlavor.legacy_default)

In [ ]:
def date_range(start, end, step=1):
    result = [start]
    while result[-1] < end:
        result.append(result[-1] + timedelta(days=step))
    return result

start_date = date.fromisoformat("2024-02-01")
end_date = date.fromisoformat("2024-07-14")
date_list = date_range(start_date, end_date)

fetch_results = []

def do_fetch(d: date):
    urls = [get_url(d, flavor) for flavor in TrackingFlavor]

    def load(u: str):
        try:
            return urllib.request.urlopen(u).read().decode("utf-8")
        except HTTPError as e:
            print(d, u, e)
            return None

    results = list(map(load, urls))

    return d, results

with ThreadPoolExecutor() as tp:
    fetch_results = list(tp.map(do_fetch, date_list))

print(fetch_results)

In [ ]:
def convert_point(d, p):
    return {
        "date": d,
        "cpu": p["cpu"] / p["count"] * 1e-3,
        "vmem": p["vmem"] * 1e-3,
        "malloc": p["malloc"] / p["count"] * 1e-3,
    }

def sum_two_points(a, b):
    if a["date"] != b["date"]:
        raise Exception("mixed dates")
    return {
        "date": a["date"],
        "cpu": a["cpu"] + b["cpu"],
        "vmem": a["vmem"] + b["vmem"],
        "malloc": a["malloc"] + b["malloc"],
    }

def null_point(d):
    return {
        "date": d,
        "cpu": None,
        "vmem": None,
        "malloc": None,
    }


def extract_athena_track_finding_point(data):
    data = data[(data["component"] == "ITkSiSpTrackFinder")]
    if len(data) == 0:
        raise Exception("not found")
    return data.iloc[0]

def extract_acts_pixel_seeding_point(data):
    data = data[((data["component"] == "ActsPixelSeedingAlg") | (data["component"] == "ActsBenchmarkSpotPixelSeedingAlg"))]
    if len(data) == 0:
        raise Exception("not found")
    return data.iloc[0]

def extract_acts_strip_seeding_point(data):
    data = data[((data["component"] == "ActsStripSeedingAlg") | (data["component"] == "ActsBenchmarkSpotStripSeedingAlg"))]
    if len(data) == 0:
        raise Exception("not found")
    return data.iloc[0]

def extract_acts_ckf_point(data):
    data = data[((data["component"] == "ActsTrackFindingAlg") | (data["component"] == "ActsBenchmarkSpotTrackFindingAlg"))]
    if len(data) == 0:
        raise Exception("not found")
    return data.iloc[0]

def extract_athena_points(d, data):
    null = null_point(d)

    try:
        track_finding_point = convert_point(d, extract_athena_track_finding_point(data))
    except Exception as e:
        track_finding_point = null
        print(d, "no track finding data")

    return {
        "track_finding": track_finding_point,
    }

def extract_acts_default_points(d, data):
    null = null_point(d)

    try:
        pixel_seeding_point = convert_point(d, extract_acts_pixel_seeding_point(data))
    except Exception as e:
        pixel_seeding_point = null
        print(d, "no pixel seeding data")

    try:
        strip_seeding_point = convert_point(d, extract_acts_strip_seeding_point(data))
    except Exception as e:
        strip_seeding_point = null
        print(d, "no strip seeding data")

    if pixel_seeding_point is not null and strip_seeding_point is not null:
        seeding_point = sum_two_points(pixel_seeding_point, strip_seeding_point)
    else:
        seeding_point = null

    try:
        ckf_point = convert_point(d, extract_acts_ckf_point(data))
    except Exception as e:
        ckf_point = null
        print(d, "no ckf data")

    if seeding_point is not null and ckf_point is not null:
        track_finding_point = sum_two_points(seeding_point, ckf_point)
    else:
        track_finding_point = null

    return {
        "pixel_seeding": pixel_seeding_point,
        "strip_seeding": strip_seeding_point,
        "seeding": seeding_point,
        "ckf": ckf_point,
        "track_finding": track_finding_point,
    }

def extract_acts_fast_points(d, data):
    null = null_point(d)

    try:
        seeding_point = convert_point(d, extract_acts_pixel_seeding_point(data))
    except Exception as e:
        seeding_point = null
        print(d, "no seeding data")

    try:
        ckf_point = convert_point(d, extract_acts_ckf_point(data))
    except Exception as e:
        ckf_point = null
        print(d, "no ckf data")

    if seeding_point is not null and ckf_point is not null:
        track_finding_point = sum_two_points(seeding_point, ckf_point)
    else:
        track_finding_point = null

    return {
        "seeding": seeding_point,
        "ckf": ckf_point,
        "track_finding": track_finding_point,
    }


def extend_data_list(list, points):
    for alg, point in points.items():
        list[alg].append(point)

In [ ]:
data_per_flavor = {flavor: defaultdict(list) for flavor in TrackingFlavor}

for d, results in fetch_results:
    legacy_default_txt, legacy_fast_txt, acts_default_txt, acts_fast_txt = results

    if legacy_default_txt is not None:
        legacy_default_spot_data = parse_spot(legacy_default_txt)
        legacy_default_spot_exec_data = legacy_default_spot_data[legacy_default_spot_data["step"] == "Execute"]

        legacy_default_points = extract_athena_points(d, legacy_default_spot_exec_data)

        extend_data_list(data_per_flavor[TrackingFlavor.legacy_default], legacy_default_points)

    if legacy_fast_txt is not None:
        legacy_fast_spot_data = parse_spot(legacy_fast_txt)
        legacy_fast_spot_exec_data = legacy_fast_spot_data[legacy_fast_spot_data["step"] == "Execute"]

        legacy_fast_points = extract_athena_points(d, legacy_fast_spot_exec_data)

        extend_data_list(data_per_flavor[TrackingFlavor.legacy_fast], legacy_fast_points)

    if acts_default_txt is not None:
        acts_default_spot_data = parse_spot(acts_default_txt)
        acts_default_spot_exec_data = acts_default_spot_data[acts_default_spot_data["step"] == "Execute"]

        acts_default_points = extract_acts_default_points(d, acts_default_spot_exec_data)

        extend_data_list(data_per_flavor[TrackingFlavor.acts_default], acts_default_points)

    if acts_fast_txt is not None:
        acts_fast_spot_data = parse_spot(acts_fast_txt)
        acts_fast_spot_exec_data = acts_fast_spot_data[acts_fast_spot_data["step"] == "Execute"]

        acts_fast_points = extract_acts_fast_points(d, acts_fast_spot_exec_data)

        extend_data_list(data_per_flavor[TrackingFlavor.acts_fast], acts_fast_points)

for flavor, data in data_per_flavor.items():
    for alg in data:
        data[alg] = pd.DataFrame(data[alg]).dropna()
        d = data[alg]

        d["date"] = pd.to_datetime(d["date"])
        d.index = d["date"]

In [ ]:
legacy_default_track_finding_data = data_per_flavor[TrackingFlavor.legacy_default]["track_finding"]
legacy_fast_track_finding_data = data_per_flavor[TrackingFlavor.legacy_fast]["track_finding"]
acts_default_track_finding_data = data_per_flavor[TrackingFlavor.acts_default]["track_finding"]
acts_fast_track_finding_data = data_per_flavor[TrackingFlavor.acts_fast]["track_finding"]

acts_fast_track_finding_data

In [ ]:
def piecewise_const(x, y, edges):
    result_x = []
    result_y = []

    for a, b in edges:
        mask = (x >= a) & (x <= b)
        result_x.append(x[mask].min())
        result_y.append(y[mask].mean())
        result_x.append(x[mask].max())
        result_y.append(y[mask].mean())

    return result_x, result_y

In [ ]:
acts_default_edges = list(map(lambda x: tuple(pd.to_datetime(date.fromisoformat(xi)) for xi in x), [
    ("2023-11-01", "2023-11-07"),
    ("2023-11-08", "2023-11-12"),
    ("2023-11-13", "2023-11-23"),
    ("2023-11-24", "2023-11-28"),
    ("2023-11-29", "2024-02-07"),
    ("2024-02-08", "2024-02-14"),
    ("2024-02-15", "2024-02-23"),
    ("2024-03-01", "2024-03-12"),
    ("2024-03-13", "2024-03-17"),
    ("2024-03-18", "2024-04-02"),
    ("2024-04-03", "2024-04-04"),
    ("2024-04-05", "2024-04-12"),
    ("2024-04-13", "2024-04-22"),
    ("2024-04-23", "2024-04-29"),
    ("2024-04-30", "2024-05-16"),
    ("2024-05-17", "2024-05-23"),
    ("2024-05-24", "2024-06-04"),
    ("2024-06-05", "2024-07-01"),
    ("2024-07-02", "2024-07-07"),
    ("2024-07-08", "2024-07-14"),
]))

acts_default_track_finding_data_fit = pd.DataFrame({
    "date": piecewise_const(acts_default_track_finding_data["date"], acts_default_track_finding_data["cpu"], acts_default_edges)[0],
    "cpu": piecewise_const(acts_default_track_finding_data["date"], acts_default_track_finding_data["cpu"], acts_default_edges)[1],
})

In [ ]:
plt.figure(figsize=(16, 9), dpi=200)

# plt.title("Track Finding CPU")
plt.ylabel("CPU time per event [s]")
plt.xticks(rotation=45, ha="right")

plt.plot(legacy_default_track_finding_data["date"], legacy_default_track_finding_data["cpu"], ".--", color="C1", label="Athena Default Track Finding")
plt.plot(legacy_fast_track_finding_data["date"], legacy_fast_track_finding_data["cpu"], ".--", color="C2", label="Athena Fast Track Finding")
plt.plot(acts_default_track_finding_data["date"], acts_default_track_finding_data["cpu"], ".", color="C3", alpha=0.5)
plt.plot(acts_default_track_finding_data_fit["date"], acts_default_track_finding_data_fit["cpu"], ".--", color="C3", label="Acts-based Default Track Finding")
plt.plot(acts_fast_track_finding_data["date"], acts_fast_track_finding_data["cpu"], ".--", color="C4", label="Acts-based Fast Track Finding")

#plt.plot(acts_seeding_data["date"], acts_seeding_data["cpu"], ".--", label="Acts-based Seeding")
#plt.plot(acts_ckf_data["date"], acts_ckf_data["cpu"], ".--", label="Acts-based CKF")

ymin, ymax = plt.ylim()
xmin, xmax = plt.xlim()
plt.xlim(xmin, xmax + (xmax-xmin)*0.03)

plt.subplots_adjust(left=0.05, right=0.99, top=0.9, bottom=0.1)

plt.legend()

plt.savefig("cpu.png")
plt.savefig("cpu.pdf")

annotations = [
    # ("2023-11-08", "!66631 - Switch ACTS\ntracking to\nAthena backends", "center", (30, -250)),
    # ("2023-11-13", "Potentially more\nEDM related changes\nlike !67034 and/or !66992", "center", (100, -350)),
    # ("2023-11-24", "!66813 - KF on seed", "center", (50, 30)),
    # ("2023-11-29", "!67323 - Aux in MTJ", "center", (50, 30)),
    ("2024-02-08", "Accidentally\ndeployed fast\ntracking config !67296", "center", (0, -70)),
    ("2024-02-15", "Restored with !68800", "center", (60, -200)),
    ("2024-03-01", "Unclear", "center", (30, 80)),
    ("2024-03-13", "Unclear", "center", (20, 40)),
    ("2024-03-18", "!69856 - Reduce branching\nfrom 3 to 1", "center", (-20, -80)),
    ("2024-04-03", "Unclear", "center", (-70, 40)),
    ("2024-04-05", "!70171 - Extend tracks\nin both directions", "center", (10, -80)),
    ("2024-04-13", "!70587\nstd::unordered_multimap\ninstead of\nboost::flat_multimap", "center", (65, 125)),
    ("2024-04-23", "!70375 - ACTS EDM\nbackend during CKF", "center", (-50, -60)),
    ("2024-04-30", "!70940 - Stop branches\non holes and outliers", "center", (20, 110)),
    ("2024-05-17", "!71469 - Bare pointers\ninstead of ElementLinks", "center", (20, 70)),
    ("2024-05-24", "!71683 - Disable\nKF on seeds", "center", (25, 30)),
    ("2024-06-05", "!71789 - Enable\nanalog clustering", "center", (25, 30)),
    ("2024-07-03", "!72610 - Faster\nmeasurement\nselector", "center", (0, 80)),
    ("2024-07-08", "!72881 - Calibrate only\nselected", "center", (-100, 0)),
]

for d, s, ha, txy in annotations: 
    d = date.fromisoformat(d)
    y = acts_default_track_finding_data.loc[d:d].cpu.iloc[0]
    t = f"{d:%Y-%m-%d}"
    plt.annotate(f"{t}\n{s}", 
                 xy=(date2num(d), y),
                 xytext=txy,
                 ha=ha,
                 textcoords="offset points",
                 arrowprops=dict(arrowstyle='-|>'))


plt.savefig("cpu_annotated.png")
plt.savefig("cpu_annotated.pdf")

plt.show();

In [ ]:
plt.figure(figsize=(16, 9))
plt.plot(legacy_default_track_finding_data["date"], legacy_default_track_finding_data["malloc"], ".--", color="C1", label="Athena Default Track Finding")
plt.plot(legacy_fast_track_finding_data["date"], legacy_fast_track_finding_data["malloc"], ".--", color="C2", label="Athena Fast Track Finding")
plt.plot(acts_default_track_finding_data["date"], acts_default_track_finding_data["malloc"], ".--", color="C3", label="Acts-based Default Track Finding")
plt.plot(acts_fast_track_finding_data["date"], acts_fast_track_finding_data["malloc"], ".--", color="C4", label="Acts-based Default Track Finding")
plt.title("Track Finding Memory")
plt.ylabel("Memory allocations [mb/event]")
plt.legend()
plt.xticks(rotation=45, ha="right");